In [1]:
from database import Database
from table import Table
from attribute import Attribute
from TableOperators import *
from DBMS_system import *

In [2]:
db = Database()
# Task I.a
# Define new tables
while True:
    new_table = create_tables()
    db.add_table(new_table)
    is_add_new_table = input("Do you still want to define a new table(yes or no)?")
    if is_add_new_table == "no":
        print('Current tables in the Database:\n')
        print(db)
        break

Please input new table name:Student
Please input at least one attribute. Input 'quit' at any time to stop.
Please input the name and type ('integer'  or 'string') for one attribute. Use ',' as delimiter :A, integer
Successfully add a new attribute
Please input the name and type ('integer'  or 'string') for one attribute. Use ',' as delimiter :B, integer
Successfully add a new attribute
Please input the name and type ('integer'  or 'string') for one attribute. Use ',' as delimiter :C, string
Successfully add a new attribute
Please input the name and type ('integer'  or 'string') for one attribute. Use ',' as delimiter :quit
The tabel Student have created successfully!
Table: Student
 | A | B | C |
---------------

Do you still want to define a new table(yes or no)?yes
Please input new table name:Grade
Please input at least one attribute. Input 'quit' at any time to stop.
Please input the name and type ('integer'  or 'string') for one attribute. Use ',' as delimiter :A, integer
Successfu

In [4]:
# Task I.b
# Ask users to input possible constraints for each defined table
for table_name, table in db.tables.items():
    print("Please input constraint for the table [" + table.name + "]. Input 'quit' at any time to stop.")
    table = create_constraint(table, db)

Please input constraint for the table [Student]. Input 'quit' at any time to stop.
Please input Boolean condition:B>17
Add boolean conditions successfully
Please input Boolean condition:quit
Please input FD:A->B
Added a new fd successfully: A->B
Please input FD:quit
The table named:  Student 
Has FD's:  ['A->B']
Do you want to remove some Fds(yes or no):no
Please input MVD:quit
Please input constraint for the table [Grade]. Input 'quit' at any time to stop.
Please input Boolean condition:quit
Please input FD:AE->G
Added a new fd successfully: AE->G
Please input FD:quit
The table named:  Grade 
Has FD's:  ['AE->G']
Do you want to remove some Fds(yes or no):no
Please input MVD:quit
Please input constraint for the table [Teacher]. Input 'quit' at any time to stop.
Please input Boolean condition:B>20
Add boolean conditions successfully
Please input Boolean condition:quit
Please input FD:A->BC
Added a new fd successfully: A->BC
Please input FD:quit
The table named:  Teacher 
Has FD's:  ['A-

In [5]:
# Task I.c
# Evaluate the NF category for the user defined DB
for table_name, table in db.tables.copy().items():
    table = update_normal_form(db, table)
    if table != None:
        db.tables[table_name] = table

This table has normal form: 1NF
Currently this table [Student] has normal form: 1NF
You can either delete this table or update its FD's.
Either type 'del' for deletion of the table or 'u' to update:u
Current FD(s) in the table: ['A->B']

Type 'a' to add a new FD or 'd' to delete a FD: a
Add a new FD: A->C
Added a new fd successfully: A->C
This table has normal form: BCNF
The table now has normal form BCNF would you like to continue (y/n)?n
This table has normal form: BCNF
Currently this table [Grade] has normal form: BCNF
This table has normal form: BCNF
Currently this table [Teacher] has normal form: BCNF


In [6]:
# Task I.d
# Ask users to define keys for each table
for table_name, table in db.tables.items():
    table.print_keys()
    table.user_define_key()
    print('Attributes in the table: ' + repr(table.attributes_names))
    #print(table.attributes)

All keys for this table are :  {'A'}
What key would you like to use? (type 'rand' to have it selected for you): rand
You have been given the key: A
Attributes in the table: {'C', 'A', 'B'}
All keys for this table are :  {'AE'}
What key would you like to use? (type 'rand' to have it selected for you): rand
You have been given the key: AE
Attributes in the table: {'E', 'A', 'G'}
All keys for this table are :  {'A'}
What key would you like to use? (type 'rand' to have it selected for you): rand
You have been given the key: A
Attributes in the table: {'C', 'A', 'B'}


In [7]:
# Task II.a
# Input new tuples to all tables
#db = test_data()
while True:
    print("The tables in current db: \n")
    print(db)
    input_str = input("Please choose a table in order to input a new tuple into it(or quit to terminate): ")
    if input_str == 'quit':
        break
    if input_str in db.tables:
        table_to_add = db.tables[input_str]
        while True:
            in_tuple_str = input("Input a new tuple into " + input_str + " using \',\' as delimiter(or quit):")
            if in_tuple_str == "quit":
                break
            in_tuple_as_list = list(in_tuple_str.split(','))
            type_list = [a.type for a in table_to_add.attributes]
            for i, el in enumerate(in_tuple_as_list):
                if type_list[i] == "integer":
                    in_tuple_as_list[i] = int(el)
            # handle case of integer
            table_to_add.add_tuple(tuple(in_tuple_as_list))

    else:
        print("Invaild input, please input again!")

The tables in current db: 

Table: Student
 | A | B | C |
---------------
Table: Grade
 | A | E | G |
---------------
Table: Teacher
 | A | B | C |
---------------

Please choose a table in order to input a new tuple into it(or quit to terminate): Student
Input a new tuple into Student using ',' as delimiter(or quit):1,18,Tony
Input a new tuple into Student using ',' as delimiter(or quit):2,21,Tom
Input a new tuple into Student using ',' as delimiter(or quit):3,21,Alex
Input a new tuple into Student using ',' as delimiter(or quit):4,24,Matt
Input a new tuple into Student using ',' as delimiter(or quit):quit
The tables in current db: 

Table: Student
 | A | B | C |
---------------
 | 1 | 18 | Tony | 
 | 2 | 21 | Tom | 
 | 3 | 21 | Alex | 
 | 4 | 24 | Matt | 
Table: Grade
 | A | E | G |
---------------
Table: Teacher
 | A | B | C |
---------------

Please choose a table in order to input a new tuple into it(or quit to terminate): Grade
Input a new tuple into Grade using ',' as delimiter(

In [8]:
# Task II.b
# Delete a tuple based on key value
while True:
    print("The tables in current db: \n", db)
    input_str = input("Would you like to delete a tuple from any table? (y/n)")
    if input_str == 'n':
        break
    else:
        table_name = input("Which table would you like to delete from?")
        table = db.tables[table_name]
        print("This the table you will be deleting from: \n", table)
        print("Which has the key: ", table.keys)
        key = input("Please provide the key for the tuple you'd like to delete: ")
        table.remove_tuple(key)


The tables in current db: 
 Table: Student
 | A | B | C |
---------------
 | 1 | 18 | Tony | 
 | 2 | 21 | Tom | 
 | 3 | 21 | Alex | 
 | 4 | 24 | Matt | 
Table: Grade
 | A | E | G |
---------------
 | 1 | 10 | A | 
 | 1 | 12 | C | 
Table: Teacher
 | A | B | C |
---------------
 | 1 | 26 | John | 

Would you like to delete a tuple from any table? (y/n)y
Which table would you like to delete from?Student
This the table you will be deleting from: 
 Table: Student
 | A | B | C |
---------------
 | 1 | 18 | Tony | 
 | 2 | 21 | Tom | 
 | 3 | 21 | Alex | 
 | 4 | 24 | Matt | 

Which has the key:  {'A'}
Please provide the key for the tuple you'd like to delete: 1
Cannot remove that tuple, foreign key error!
The tables in current db: 
 Table: Student
 | A | B | C |
---------------
 | 1 | 18 | Tony | 
 | 2 | 21 | Tom | 
 | 3 | 21 | Alex | 
 | 4 | 24 | Matt | 
Table: Grade
 | A | E | G |
---------------
 | 1 | 10 | A | 
 | 1 | 12 | C | 
Table: Teacher
 | A | B | C |
---------------
 | 1 | 26 | John 

In [ ]:
def test_data():
    A = Attribute("A", "integer")
    B = Attribute("B", "integer")
    C = Attribute("C", "string")
    D = Attribute("D", "string")
    E = Attribute("E", "integer")
    F = Attribute("F", "string")
    G = Attribute("G", "string")

    new_table1 = Table("Student", [A,B,C])
    new_table2 = Table("Grade", [E, G])
    new_table3 = Table("test3", [A,B,C])

    new_table1.add_fd("A->B")
    new_table1.add_fd("A->C")
    new_table1.master_key = "A"
    new_table1.add_tuple((1,18,"Tony"))
    new_table1.add_tuple((2,22,"Tom"))
    new_table1.add_tuple((3,21,"Alex"))
    new_table1.add_tuple((4,24,"Matt"))

    new_table2.add_fd("E->G")
    new_table2.master_key = "E"
    new_table2.add_tuple((10,"A"))
    new_table2.add_tuple((12,"B"))

    new_table3.add_fd("AB->C")
    new_table3.master_key = "AB"
    new_table3.add_tuple((1,10,"A"))
    new_table3.add_tuple((1,12,"B"))

    
    db = Database()
    db.add_table(new_table1)
    db.add_table(new_table2)
    db.add_table(new_table3)

    return db
db = test_data()

In [9]:
# Task II.c
# Find tuples based on conditions
while True:
    print("The tables in current db: \n", db)
    input_str = input("Would you like to find any tuples from some conditions in the tables (y/n)?")
    if input_str == 'n':
        break
    else:
        table_name = input("Which table would you like to search in? ")
        table = db.tables[table_name]
        attr_name = input("Which attribute would you like to condition by? ")
        values = list(input("Which values would you like to condition on? ").split())
        conditions = list(input("Which condition do you want (>, >, <>, or ==): ").split())
        tuple_key = list(table.get_tuple_conditions(attr_name, values, conditions).values())
        keys = [key for key in tuple_key[0]]
        print()
        print('The resulting tuple(s):')
        for i in keys:
            print(table.tuples[i])
        print('\n')

# Group by attribute(s) in specific table
while True:
    print("The tables in current db: \n", db)
    input_str = input("Would you like to group by any attributes from the tables (y/n)?")
    if input_str == 'n':
        break
    else:
        table_name = input("Which table would you like to group by in?")
        table = db.tables[table_name]
        attr_name = input("Which attribute would you like to group by?")
        table.group_by(attr_name)

The tables in current db: 
 Table: Student
 | A | B | C |
---------------
 | 1 | 18 | Tony | 
 | 2 | 21 | Tom | 
 | 3 | 21 | Alex | 
 | 4 | 24 | Matt | 
Table: Grade
 | A | E | G |
---------------
 | 1 | 10 | A | 
 | 1 | 12 | C | 
Table: Teacher
 | A | B | C |
---------------
 | 1 | 26 | John | 

Would you like to find any tuples from some conditions in the tables (y/n)?y
Which table would you like to search in? Student
Which attribute would you like to condition by? B
Which values would you like to condition on? 20
Which condition do you want (>, >, <>, or ==): >

The resulting tuple(s):
(4, 24, 'Matt')
(2, 21, 'Tom')
(3, 21, 'Alex')


The tables in current db: 
 Table: Student
 | A | B | C |
---------------
 | 1 | 18 | Tony | 
 | 2 | 21 | Tom | 
 | 3 | 21 | Alex | 
 | 4 | 24 | Matt | 
Table: Grade
 | A | E | G |
---------------
 | 1 | 10 | A | 
 | 1 | 12 | C | 
Table: Teacher
 | A | B | C |
---------------
 | 1 | 26 | John | 

Would you like to find any tuples from some conditions in

ValueError: '1' is not in list

In [12]:
# TODO Task II.d
# Choose to perform the operators for two tables
while True:
    print("The tables in current db: \n", db)
    input_str = input("Would you like to perform operators on tables (y/n)?")
    if input_str == 'n':
        break
    else:
        table_one = input("Select the first table: ")
        table1 = db.tables[table_one]
        table_two = input("Select the second table: ")
        table2 = db.tables[table_two]
        while True:
            input_str = input('Select operators:\
            \n \'cjoin\' for cross join\
                              \n \'njoin\' for natural join\
                              \n \'union\' for union\
                              \n \'inter\' for intersection\
                              \n \'diff\' for difference\n')
            print('Result:')
            if input_str == 'njoin':
                print(njoin_table(table1,table2))
            if input_str == 'cjoin':
                print(cjoin_table(table1,table2))
            if input_str == 'union':
                print(union(table1,table2))
            if input_str == 'inter':
                print(intersection(table1,table2))
            if input_str == 'diff':
                print(difference(table1,table2))
            else:
                break
            print(=============================)

The tables in current db: 
 Table: Student
 | A | B | C |
---------------
 | 1 | 18 | Tony | 
 | 2 | 21 | Tom | 
 | 3 | 21 | Alex | 
 | 4 | 24 | Matt | 
Table: Grade
 | A | E | G |
---------------
 | 1 | 10 | A | 
 | 1 | 12 | C | 
Table: Teacher
 | A | B | C |
---------------
 | 1 | 26 | John | 

Would you like to perform operators on tables (y/n)?y
Select the first table: Student
Select the second table: Teacher
Select operators:            
 'cjoin' for cross join                              
 'njoin' for natural join                              
 'union' for union                              
 'inter' for intersection                              
 'diff' for difference
diff

   A   B     C
0  1  18  Tony
1  2  21   Tom
2  3  21  Alex
3  4  24  Matt

Select operators:            
 'cjoin' for cross join                              
 'njoin' for natural join                              
 'union' for union                              
 'inter' for intersection                   

In [13]:
# TODO Task II.e
# Delete a table and ensure the across-table integrity
print([name for name, _ in db.tables.items()])
table_to_del = input("What is the name of the table you'd like to delete?")
db.delete_table(table_to_del)

['Student', 'Grade', 'Teacher']
What is the name of the table you'd like to delete?Student
You cannot delete this table because its attributes are shared by Grade


False

In [ ]:
# Write database to text file
write_ans = input("Would you like to write whole database (\'all\') or a specific table to a txt file?")
if write_ans == "all":
    db.write_database()
elif write_ans == 'n':
    print("Nothing")
else:
    if write_ans in db.tables:
        db.write_table(write_ans)
    else:
        print("There is not table with name: " + write_ans)